In [1]:
import tensorflow as tf
import keras
import jieba
import pickle as pkl
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.python.layers import core as layers_core
import numpy as np
import nltk

Using TensorFlow backend.


In [2]:
def data_preprocess(path):
    corpus_en = []
    corpus_ch = []   
    with open(path, 'r',encoding='utf-8') as f:
        for i, line in enumerate(f): # i从0开始
            line = line.strip()
#             .decode('utf8')
            if i%2 ==0:
                corpus_en.append(line)
            else:
                corpus_ch.append(line)
                
    corpus_en = '\n'.join(corpus_en)
    corpus_ch = '\n'.join(corpus_ch)
    return corpus_en, corpus_ch

train_en, train_ch = data_preprocess("data/data_origin/train.txt")
valid_en, valid_ch = data_preprocess("data/data_origin/valid.txt")

def segment(corpus, tokenizer, savepath=None):
    tokenized_corpus = []
    tokenized_corpus = ' '.join([_ for _ in tokenizer(corpus) if _.strip(' ')])
    tokenized_corpus = tokenized_corpus.split(' \n ')
    if savepath:
        with open(savepath,'wb+') as fw:
            pkl.dump(tokenized_corpus, fw)
    return tokenized_corpus

train_en = segment(train_en, jieba.cut)
train_ch = segment(train_ch, lambda k: iter(k.strip()))
valid_en = segment(valid_en, jieba.cut)
valid_ch = segment(valid_ch, lambda k: iter(k.strip())) #暂时分字

def vocab(data, topK=None):
    word2id = Counter() # Counter是dict的一个子类
    for sentence in data:
        for word in sentence.split():
            word2id[word] += 1
    word2id = word2id.most_common() #按频率从高到低排序
    if topK:
        word2id = word2id[:topK]
    word2id, _ = zip(*word2id)
    word2id = {word : i + 4 for i, word in enumerate(word2id)}  # 将0，1，2，3的位置空出来
    word2id['<PAD>'] = 0 # 中止id
    word2id['<UNK>'] = 1 # 字典中没有该字/词的id表示
    word2id['<S>'] = 2 # 起始id
    word2id['</S>'] = 3
    id2word = dict(zip(word2id.values(), word2id.keys()))
    return word2id, id2word

en_word2id, en_id2word = vocab(train_en+valid_en, 60000) # 验证集的词不应该加入字典 去点valid_en
ch_word2id, ch_id2word = vocab(train_ch+valid_ch, 8800)

# 把数据中的字/词转成对应id
print(len(en_word2id))
print(len(ch_word2id))

def transform(data, word2id):
    ret_data = []
    for sentence in data:
        ret_data.append([word2id.get(word, 1) for word in sentence.split()]) # word2id.get(word, 1) 如果word不在字典中，则返回默认值1
    return ret_data


train_en_corpus = transform(train_en, en_word2id)
train_ch_corpus = transform(train_ch, ch_word2id)
valid_en_corpus = transform(valid_en, en_word2id)
valid_ch_corpus = transform(valid_ch, ch_word2id)

with open(r'data/data_origin/vocab_dict_and_corpus_origin.pkl', 'wb+') as fw:
    pkl.dump([en_word2id, en_id2word, ch_word2id, ch_id2word, train_en_corpus, train_ch_corpus, valid_en_corpus, valid_ch_corpus], fw)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\19843\AppData\Local\Temp\jieba.cache
Loading model cost 2.256 seconds.
Prefix dict has been built succesfully.


57325
4882


In [10]:
def buildDict(path):
    corpus_en = [] 
    corpus_ch = []
    corpusDict = {}
    with open(path, 'r',encoding='utf-8') as f:
        for i, line in enumerate(f): # i从0开始
            line = line.strip()
            if i%2==0:
                corpus_en.append(line)
            else:
                corpus_ch.append(line)
    for i in range(len(corpus_ch)):
        corpusDict[corpus_en[i]] = corpus_ch[i]
    return corpusDict
corpusDict = buildDict(r'C:\Users\19843\Desktop\natural_language_processing\Experiment5\UM-Corpus\data\Bilingual\Education\Bi-Education.txt')

# 读入
def data_preprocess(path):
    corpus_en = []
    with open(path, 'r',encoding='utf-8') as f:
        for i, line in enumerate(f): # i从0开始
            if i%2 ==0:
                line = line.strip()
                corpus_en.append(line)
    return corpus_en

testPath = r'C:\Users\19843\Desktop\natural_language_processing\Experiment5\bleu_eval\train.txt'
test_en = data_preprocess(testPath)

def predict_wR(test_en_wR):
    pred = []
    count = 0
    for s in test_en_wR:
        try:
            pred.append(corpusDict[s])
        except:
#             tf.reset_default_graph()
#             p = predict(s)
#             pred.append(p[:p.find('<PAD>')])
              count += 1
              pred.append('find nothing')
    print(count)
    return pred

# t1 = time.time()
pred2 = predict_wR(test_en)
# t2 = time.time()
# print(t2-t1)

0
